# Keyword-Extraction using BERT-base-multilingual-cased

Use BERT Token Classification Model to extract keyword tokens from a sentence.

## Prepare Dataset for BERT.


In [ ]:
!pip install transformers

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 569.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=df8d3f09a2b932ca9caac26228e7a0b7b3354378759ec9593018ee91b3d8d52b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 58.6 MB/s eta 0:00:00


In [ ]:
import os
from underthesea import sent_tokenize


In [ ]:
limits= {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "can not",
"cannot":" can not",
"can't've": "can not have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"dont":"do not" ,
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have",
"other's":"others",
"she’s": "she is",
"can’t": "can not"
}

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = "./drive/MyDrive/BERT-Keyword-Extractor/maui-semeval-train"

In [ ]:
txt = sorted([f for f in os.listdir(train_path) if not f.endswith("-justTitle.txt") and not f.endswith(".key") and not f.endswith("-CrowdCountskey")])
key = sorted([f for f in os.listdir(train_path) if f.endswith(".key")])

In [ ]:
filekey = dict()
for i, k in enumerate(txt):
    filekey[key[i]] = k

In [ ]:
def convert(key):
    sentences = ""
    for line in open(train_path + "/" + filekey[key], 'r'):
        sentences += (" " + line.rstrip())
    tokens = sent_tokenize(sentences)
    key_file = open(train_path + "/" + str(key),'r')
    keys = [line.strip() for line in key_file]
    key_sent = []
    labels = []

    for token in tokens:

        for word in token.split():

            if word.lower() in limits:
                token = token.replace(word, limits[word.lower()])

        punctuations = '''!()-[]{};:\,<>'"./?@#$%^&*_~'''



        no_punct = ""
        for char in token:
            if char not in punctuations:
                no_punct = no_punct + char



        token=no_punct

        z = ['O'] * len(token.split())
        for k in keys:

            if k in token.lower():

                if len(k.split())==1:
                    try:

                        z[token.lower().split().index(k.lower().split()[0])] = 'B'

                    except ValueError:
                        continue


                elif len(k.split())>1:

                    try:
                        h = k.split()
                        def index(arr, num):
                            a = []
                            for i, x in enumerate(arr):
                                if x == num:
                                    a.append(i)
                            return(a)




                        lis1 = index(token.lower().split(),k.lower().split()[0])
                        lis2 =  index(token.lower().split(),k.lower().split()[1])
                        x = []
                        for i in lis1:
                            for j in lis2:
                                if abs(i - j) == 1:
                                    x.append(i)
                        num = x[0]
                        z[num] = 'B'

                        for j in range(1, len(h)):
                            z[num + j] = 'I'

                    except:
                        continue

            elif k.lower().split()[0] in token.lower():

                    try:
                        h = k.split()
#                         print(token)
#                         print(len(token.split()))
                        def index(arr, num):
                            a = []
                            for i, x in enumerate(arr):
                                if x == num:
                                    a.append(i)
                            return(a)




                        lis1 = index(token.lower().split(),k.lower().split()[0])
                        lis2 =  index(token.lower().split(),k.lower().split()[1])
                        x = []
                        for i in lis1:
                            for j in lis2:
                                if abs(i - j) == 1:
                                    x.append(i)

                        num = x[0]
                        z[num] = 'B'
                        p= len(h)-1
                        y=0
                        for j in range(1,len(token.split())):
                            if (k.lower().split()[j] in token.lower().split()):
#                                 print('rahul')
                                print(j)
                                z[num + j] = 'I'
                                y+=1
                                if y==p:
                                  break
                            else:
                                continue

                    except:
                        continue

        for m, n in enumerate(z):
            if z[m] == 'I' and z[m-1] == 'O':
                z[m] = 'O'

        if set(z) != {'O'}:
            labels.append(z)
            key_sent.append(token)
    return key_sent, labels

In [ ]:
sentences_ = []
labels_ = []
for key, value in filekey.items():
    s, l = convert(key)
    sentences_.append(s)
    labels_.append(l)
sentences = [item for sublist in sentences_ for item in sublist]
labels = [item for sublist in labels_ for item in sublist]
# print(len(sentences), len(labels))

Streaming output truncated to the last 5000 lines.
67
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
1
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
1
2
1
1
2
3
4
1
2
1
2
3
4
5
6
7
8
1
2
1
2
3
4
1
7
8
9
10
1
1
1
2
3
4
5
1
1
2
3
4
5
1
2
3
1
2
3
4
5
1
1
2
1
2
3
4
1
3
4
1
2
4
5
1
1
1
2
3
6
7
8
11
13
15
17
19
20
21
30
1
2
3
4
1
2
3
1
2
1
2
1
2
3
1
2
1
2
1
2
1
1
2
1
2
1
3
4
1
3
4
1
3
4
1
2
3
4
5
6
7
8
9
11
12
1
2
1
3
1
3
1
3
4
1
2
1
2
1
1
3
1
3
1
2
11
12
1
4
1
2
3
4
1
4
1
2
3
1
2
3
4
5
6
7
8
1
1
2
3
4
5
6
7
8
9
10
1
2
1
2
1
4
5
6
7
8
9
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
10
11
1
1
2
1
1
3
4
6
7
9
11
12
14
15
17
19
22
1
3
4
5
10
13
14
15
32
1
3
1
2
1
1
1
2
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
2
3
4
5
6
7
1
2
1
2
1
1
1
2
3
4
1
2
3
4
1
2
1
1
2
3
4
5
6
7
1
2
1
1
2
1
2
3
1
1
2
1
2
3
1
1
2
1
2
4
5
1
2
1
1
2
3
1
3
5
8
9
1
7
8
9
11
22
1
1
1
3
1
1
1
2
1
1
3
4
5
1
3
1
1
2
3
1
2
1
2
3
1
2
1
1
2
3
1
1
2
3
4
5
6
1
2
4
7
8
10
11
13
15
17
18
19
21
22
24
25
27
28
35
36
38
1
17
18
24
25
27
28
29
30
31
33
34
35
1
2
3
1
1


## Model

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import trange
from transformers import BertTokenizer, BertForTokenClassification


In [ ]:
MAX_LEN = 128
bs = 32

In [ ]:
tag2idx = {'B': 0, 'I': 1, 'O': 2}
tags_vals = ['B', 'I', 'O']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
model_path = "minhtu0408/BERT-keyword-extractor"


In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=False)

In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
model = BertForTokenClassification.from_pretrained(model_path, num_labels=len(tag2idx))

In [ ]:
model = model.to(device)

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=5e-5)

In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
epochs = 5
max_grad_norm = 1.0
best_valid_accuracy = 0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        tk_loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        tk_loss.loss.backward()
        # track train loss
        tr_loss += tk_loss.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_loss += tmp_eval_loss.loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
    valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.007362911944576868
Validation loss: 0.05762857270760918
Validation Accuracy: 0.9868916445558177


Epoch:  20%|██        | 1/5 [06:02<24:09, 362.44s/it]

F1-Score: 0.5667311411992264
Train loss: 0.006760054495728065
Validation loss: 0.05949687251364285
Validation Accuracy: 0.9865569108687107


Epoch:  40%|████      | 2/5 [12:06<18:10, 363.60s/it]

...

F1-Score: 0.5612754514022282
Train loss: 0.005944859482907645
Validation loss: 0.06085654107396895
Validation Accuracy: 0.9865093111242139


Epoch:  80%|████████  | 4/5 [24:15<06:04, 364.28s/it]

F1-Score: 0.5689171498036282
Train loss: 0.005458818412268728
Validation loss: 0.0618676049459093
Validation Accuracy: 0.9862754070034068


Epoch: 100%|██████████| 5/5 [30:20<00:00, 364.13s/it]

F1-Score: 0.5761886933732685


###Save Model

In [ ]:
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./drive/MyDrive/BERT-Keyword-Extractor/TextToKeywordsModel/tokenizer_config.json',
 './drive/MyDrive/BERT-Keyword-Extractor/TextToKeywordsModel/special_tokens_map.json',
 './drive/MyDrive/BERT-Keyword-Extractor/TextToKeywordsModel/vocab.txt',
 './drive/MyDrive/BERT-Keyword-Extractor/TextToKeywordsModel/added_tokens.json')

### Get keywords from sentence

In [ ]:
def keywordextract(sentence):
    tags_vals = ['0', '1', '2']
    tkns = tokenizer.tokenize(sentence)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model.eval()
    prediction = []
    with torch.no_grad():
      logits = model(tokens_tensor).logits
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
    pred_tags = [tags_vals[p_i] for p_i in predictions[0]]
    original_tokens = tokenizer.convert_ids_to_tokens(indexed_tokens)
    merged_tokens = []
    for orig_token, tag in zip(original_tokens, pred_tags):
        if orig_token.startswith("##"):
            merged_tokens[-1] += orig_token[2:]
        else:
            merged_tokens.append(orig_token)
    keywords = []
    current_keyword = ""
    for token, tag in zip(merged_tokens, pred_tags):
        if tag == '0':
            if current_keyword:
                keywords.append(current_keyword)
            current_keyword = token
        elif tag == '1':
            current_keyword += " " + token
    if current_keyword:
        keywords.append(current_keyword)
    return keywords

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)

    logits = logits.logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



In [ ]:
# # Example sử dụng:
text = "Lập trình hướng đối tượng (OOP) là một phương pháp lập trình mạnh mẽ và linh hoạt, được xây dựng trên các nguyên tắc cơ bản như đóng gói, kế thừa và đa hình. Trong OOP, chương trình được tổ chức thành các đối tượng, mỗi đối tượng đại diện cho một thực thể có thể tồn tại trong thế giới thực. Các đối tượng này tương tác thông qua các phương thức, là các hàm hoặc thủ tục được đặc biệt liên kết với từng đối tượng."
extracted_keywords = keywordextract(text)
print("Keywords:", extracted_keywords)


['0', '1', '1', '1', '1', '2', '2', '2', '2', '2', '0', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '0', '1', '1', '0', '1', '0', '1', '2', '2', '0', '1', '2', '2', '2', '2', '2', '2', '2', '0', '1', '2', '2', '0', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2']
Keywords: ['Lập trình hướng đối tượng', 'phương pháp lập trình mạnh mẽ', 'nguyên tắc cơ', 'bản như', 'đóng gói', 'thừa và', 'trình được', 'thành']
